Corroborar si este es mas rapido al no seleccionar días separados

In [4]:
# Obtener última fecha registrada en la tabla
import pandas as pd
import xarray as xr
import sqlite3
from pathlib import Path
import os
import sqlite3

import cdsapi
import zipfile
from netCDF4 import Dataset, num2date


def definir_star_end(DB_PATH,start_="",end_=""):

    if start_=="":

        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        cursor.execute("SELECT MAX(date) FROM climate_test")
        row = cursor.fetchone()
        conn.close()

        # Si hay datos, usar la fecha siguiente como inicio; si no, arrancar manualmente
        if row and row[0]:
            start = pd.to_datetime(row[0]) + pd.Timedelta(days=1)
        else:
            start = pd.Timestamp("2023-01-01")  # o cualquier fecha de inicio deseada
    else:
        start = pd.Timestamp(start_)
    if end_=="":
    # Siempre terminar en ayer
        end = pd.Timestamp("today").normalize() - pd.Timedelta(days=1)
    else:
        end = pd.Timestamp(end_)       

    return (start,end)

# Generar fechas faltantes
start_,end_="2021-01-01","2022-12-31"

BASE_DIR = Path().resolve()  

DB_PATH = BASE_DIR.parent / "backend" / "data" / "mi_base_de_datos5.db"

start,end=definir_star_end(DB_PATH,start_,end_)


dates = pd.date_range(start, end, freq="D")
periods = sorted(dates.to_series().dt.to_period("M").unique())

print(f"Iniciando descarga desde {start.date()} hasta {end.date()}")






Iniciando descarga desde 2021-01-01 hasta 2022-12-31


In [5]:


TMP_DIR   = "Clima\\tmp_test20d"
CDS_URL   = 'https://cds.climate.copernicus.eu/api'
CDS_KEY   = 'd9af180c-f7f8-4a2e-8029-09018fb8c920'
AREA      = [-32.0, -63.5, -33.0, -61.0]  # [Norte, Oeste, Sur, Este]
AREA      = [-33.0, -63.25, -33.0, -63.25]  # [Norte, Oeste, Sur, Este]
VARIABLES = ['2m_temperature','2m_dewpoint_temperature','total_precipitation']
TMP_DIR = os.path.abspath("./tmp_cds")
os.makedirs(TMP_DIR, exist_ok=True)

# ————— Prepara SQLite y tabla de prueba —————
conn = sqlite3.connect(DB_PATH)
cur  = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS climate_test (
    date TEXT,
    latitude REAL,
    longitude REAL,
    t2m REAL,
    d2m REAL,
    tp REAL,
    PRIMARY KEY(date, latitude, longitude)
)
""")
conn.commit()

# # ————— Calcula últimos 20 días —————
# end   = pd.to_datetime("today").normalize() - pd.Timedelta(days=1)
# start = end - pd.Timedelta(days=19)

In [6]:

dates = pd.date_range(start, end, freq="D")
years  = sorted({str(d.year)     for d in dates})
months = sorted({f"{d.month:02d}" for d in dates})
days   = sorted({f"{d.day:02d}"   for d in dates})

print(f"Descargando datos de {start.date()} a {end.date()} ({len(dates)} días)…")

# ————— Descarga del ZIP con los 3 *.nc —————
zip_path = os.path.join(TMP_DIR, "test20d.zip")
c = cdsapi.Client(url=CDS_URL, key=CDS_KEY)


try:
    c.retrieve(
        'derived-era5-single-levels-daily-statistics',
        {
            'product_type':    'reanalysis',
            'format':          'netcdf',       # el API nos devuelve un ZIP
            'variable':        VARIABLES,
            'year':            years,
            'month':           months,
            'day':             days,
            'daily_statistic': 'daily_mean',
            'frequency':       '6_hourly',
            'time_zone':       'utc+00:00',
            'area':            AREA,
        },
        zip_path
    )

    import tempfile

    def ingest_from_zip(table_name, zip_filepath):
        """Extrae cada .nc a un NamedTemporaryFile y vuelca a SQLite."""
        extracted_paths = []

        # 1) Abre el ZIP
        with zipfile.ZipFile(zip_filepath, 'r') as z:
            nc_members = [m for m in z.namelist() if m.endswith('.nc')]
            if not nc_members:
                raise RuntimeError("No encontré archivos .nc en el ZIP")

            for member in nc_members:
                # crea un temp file garantizado escribible
                tmpf = tempfile.NamedTemporaryFile(suffix=".nc", delete=False)
                # vuelca el contenido
                with z.open(member) as src:
                    tmpf.write(src.read())
                tmpf.close()
                extracted_paths.append(tmpf.name)

        # 2) Lee coordenadas y variables
        data_arrays = {}
        time_objs = latitudes = longitudes = None

        for nc_path in extracted_paths:
            ds = Dataset(nc_path)
            # detecta nombre de dim/vars
            if time_objs is None:
                time_var = next(v for v in ds.variables if "time" in v.lower())
                lat_var  = next(v for v in ds.variables if "lat"  in v.lower())
                lon_var  = next(v for v in ds.variables if "lon"  in v.lower())
                times    = ds.variables[time_var][:]
                time_objs= num2date(times,
                            ds.variables[time_var].units,
                            getattr(ds.variables[time_var], 'calendar', 'standard'))
                latitudes = ds.variables[lat_var][:]
                longitudes= ds.variables[lon_var][:]
            # identifica la variable de datos
            var_keys = [v for v in ds.variables 
                        if v not in (time_var, lat_var, lon_var, "number")]
            if len(var_keys) != 1:
                raise RuntimeError(f"Variables inesperadas en {nc_path}: {var_keys}")
            var = var_keys[0]
            data_arrays[var] = ds.variables[var][:]
            ds.close()

        # 3) Inserta en SQLite
        sql = f"""INSERT OR REPLACE INTO {table_name}
                (date, latitude, longitude, t2m, d2m, tp)
                VALUES (?, ?, ?, ?, ?, ?)"""
        cnt = 0
        for ti, dt in enumerate(time_objs):
            # usa strftime directamente en el objeto cftime
            date_str = dt.strftime("%Y-%m-%d")
            for yi, lat in enumerate(latitudes):
                for xi, lon in enumerate(longitudes):
                    v_t2m = float(data_arrays.get("t2m", data_arrays.get("2m_temperature"))[ti,yi,xi])
                    v_d2m = float(data_arrays.get("d2m", data_arrays.get("2m_dewpoint_temperature"))[ti,yi,xi])
                    v_tp  = float(data_arrays.get("tp",  data_arrays.get("total_precipitation"))[ti,yi,xi])
                    cur.execute(sql, (date_str, float(lat), float(lon), v_t2m, v_d2m, v_tp))
                    cnt += 1
        conn.commit()

        # 4) Borra los temporales
        for p in extracted_paths:
            try:
                os.remove(p)
            except OSError:
                pass
        try:
            os.remove(zip_filepath)
        except OSError:
            pass

        return cnt


    # ————— Ejecuta ingesta y reporta —————
    inserted = ingest_from_zip("climate_test", zip_path)
    print(f"✔ Insertadas/actualizadas {inserted} filas en climate_test")
except:
    pass
# conn.close()


Descargando datos de 2021-01-01 a 2022-12-31 (730 días)…


2025-09-25 12:01:02,351 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-25 12:01:02,352 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os, math, sqlite3, zipfile, tarfile, tempfile, shutil
import numpy as np
import pandas as pd
from netCDF4 import Dataset, num2date
import cdsapi

# ========= CONFIG =========
# START = "2023-01-01"                 # <-- cambia
# END   = "2024-12-31"                 # <-- cambia
# AREA  = [-32.0, -63.5, -33.0, -61.0]     # N, W, S, E  (ejemplo Iberia) <-- cambia


# DB_PATH = os.path.join(TMP_DIR, "climate.sqlite")
TABLE   = "climate_wxt2"              # nueva tabla
LATLON_DECIMALS = 6
# ==========================

os.makedirs(TMP_DIR, exist_ok=True)
# c = cdsapi.Client()

# rango de fechas
start = pd.to_datetime(start_); end = pd.to_datetime(end_)
dates  = pd.date_range(start, end, freq="D")
years  = sorted({str(d.year)      for d in dates})
months = sorted({f"{d.month:02d}" for d in dates})
days   = sorted({f"{d.day:02d}"   for d in dates})
print(f"Descargando datos de {start.date()} a {end.date()} ({len(dates)} días)…")

# SQLite
conn = sqlite3.connect(DB_PATH)
cur  = conn.cursor()
cur.execute(f"""
CREATE TABLE IF NOT EXISTS {TABLE} (
    date TEXT,
    latitude REAL,
    longitude REAL,
    t2m_min REAL,
    t2m_max REAL,
    ws10 REAL,
    PRIMARY KEY (date, latitude, longitude)
)
""")
conn.commit()

# helpers
def to_sql(v):
    try:
        v = float(v)
    except (TypeError, ValueError):
        return None
    return v if math.isfinite(v) else None

def date_to_str(dt):
    # válido para datetime y cftime
    return f"{int(dt.year):04d}-{int(dt.month):02d}-{int(dt.day):02d}"

def is_container(p): return zipfile.is_zipfile(p) or tarfile.is_tarfile(p)

def extract_nc_members(container_path):
    out = []
    if zipfile.is_zipfile(container_path):
        with zipfile.ZipFile(container_path, 'r') as z:
            for m in z.namelist():
                if m.endswith(".nc"):
                    tmpf = tempfile.NamedTemporaryFile(suffix=".nc", delete=False, dir=TMP_DIR)
                    with z.open(m) as src: shutil.copyfileobj(src, tmpf)
                    tmpf.close(); out.append(tmpf.name)
    elif tarfile.is_tarfile(container_path):
        with tarfile.open(container_path, 'r:*') as t:
            for m in t.getmembers():
                if m.name.endswith(".nc"):
                    f = t.extractfile(m)
                    if not f: continue
                    tmpf = tempfile.NamedTemporaryFile(suffix=".nc", delete=False, dir=TMP_DIR)
                    shutil.copyfileobj(f, tmpf)
                    tmpf.close(); out.append(tmpf.name)
    return out


Descargando datos de 2021-01-01 a 2022-12-31 (730 días)…


In [9]:

def ingest_from_zip(table_name, download_path, kind):
    extracted_paths = extract_nc_members(download_path) if is_container(download_path) else [download_path]
    data_arrays = {}
    time_objs = latitudes = longitudes = None
    time_var = lat_var = lon_var = None

    for nc_path in extracted_paths:
        ds = Dataset(nc_path)
        if time_objs is None:
            time_var = next(v for v in ds.variables if "time" in v.lower())
            lat_var  = next(v for v in ds.variables if "lat"  in v.lower())
            lon_var  = next(v for v in ds.variables if "lon"  in v.lower())
            times = ds.variables[time_var][:]
            units = ds.variables[time_var].units
            cal   = getattr(ds.variables[time_var], 'calendar', 'standard')
            try:
                time_objs = num2date(times, units, calendar=cal, only_use_cftime_datetimes=False)
            except TypeError:
                time_objs = num2date(times, units, calendar=cal)
            latitudes  = ds.variables[lat_var][:]
            longitudes = ds.variables[lon_var][:]
        for v in ds.variables:
            if v in (time_var, lat_var, lon_var, "number"): continue
            if v not in data_arrays:
                data_arrays[v] = ds.variables[v][:]
        ds.close()

    def pick(*names):
        for n in names:
            a = data_arrays.get(n)
            if a is not None: return a
        return None

    A_t2m = pick('t2m', '2m_temperature')
    A_u10 = pick('u10', '10m_u_component_of_wind')
    A_v10 = pick('v10', '10m_v_component_of_wind')
    A_ws10 = np.sqrt(A_u10**2 + A_v10**2) if (A_u10 is not None and A_v10 is not None) else None

    sql = f"""
    INSERT INTO {table_name}
      (date, latitude, longitude, t2m_min, t2m_max, ws10)
    VALUES (?, ?, ?, ?, ?, ?)
    ON CONFLICT(date, latitude, longitude) DO UPDATE SET
      t2m_min = CASE WHEN excluded.t2m_min IS NOT NULL AND excluded.t2m_min = excluded.t2m_min
                     THEN excluded.t2m_min ELSE {table_name}.t2m_min END,
      t2m_max = CASE WHEN excluded.t2m_max IS NOT NULL AND excluded.t2m_max = excluded.t2m_max
                     THEN excluded.t2m_max ELSE {table_name}.t2m_max END,
      ws10    = CASE WHEN excluded.ws10    IS NOT NULL AND excluded.ws10    = excluded.ws10
                     THEN excluded.ws10    ELSE {table_name}.ws10 END
    """

    cnt = 0
    for ti, dt in enumerate(time_objs):
        date_str = date_to_str(dt)
        for yi, lat in enumerate(latitudes):
            latf = round(float(lat), LATLON_DECIMALS)
            for xi, lon in enumerate(longitudes):
                lonf = round(float(lon), LATLON_DECIMALS)
                t2m_min = to_sql(A_t2m[ti, yi, xi]) if (kind == "tmin" and A_t2m is not None) else None
                t2m_max = to_sql(A_t2m[ti, yi, xi]) if (kind == "tmax" and A_t2m is not None) else None
                ws10    = to_sql(A_ws10[ti, yi, xi]) if (kind == "mean" and A_ws10 is not None) else None
                cur.execute(sql, (date_str, latf, lonf, t2m_min, t2m_max, ws10))
                cnt += 1
    conn.commit()

    for p in extracted_paths:
        if p != download_path:
            try: os.remove(p)
            except OSError: pass
    return cnt

# requests (mean->wind; tmin; tmax)
V_WIND = ['10m_u_component_of_wind', '10m_v_component_of_wind']
V_T    = ['2m_temperature']
REQS = [
    ("mean", "daily_mean",    V_WIND),
    ("tmin", "daily_minimum", V_T),
    ("tmax", "daily_maximum", V_T),
]

# chunk mensual
def iter_month_chunks(s, e):
    s = pd.to_datetime(s).normalize()
    e = pd.to_datetime(e).normalize()
    cur = s
    while cur <= e:
        end_m = (cur + pd.offsets.MonthEnd(0))
        yield cur, min(e, end_m)
        cur = end_m + pd.Timedelta(days=1)

def ymd_lists(s, e):
    d = pd.date_range(s, e, freq="D")
    years  = sorted({str(x.year)      for x in d})
    months = sorted({f"{x.month:02d}" for x in d})
    days   = sorted({f"{x.day:02d}"   for x in d})
    return years, months, days

# ejecutar por chunks
total = 0
print(f"Procesando por meses de {start_} a {end_}…")
for cs, ce in iter_month_chunks(start, end):
    years, months, days = ymd_lists(cs, ce)
    print(f"  -> {cs.date()} a {ce.date()}")
    for kind, stat, variables in REQS:
        out_path = os.path.join(TMP_DIR, f"{kind}_{cs:%Y%m%d}_{ce:%Y%m%d}.nc")
        c.retrieve(
            'derived-era5-single-levels-daily-statistics',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': variables,
                'year': years,
                'month': months,
                'day': days,
                'daily_statistic': stat,
                'frequency': '6_hourly',
                'time_zone': 'utc+00:00',
                'area': AREA,
            },
            out_path
        )
        total += ingest_from_zip(TABLE, out_path, kind)
        try: os.remove(out_path)
        except OSError: pass

print(f"✔ Insertadas/actualizadas {total} filas en {TABLE} (DB: {DB_PATH})")
conn.close()


Procesando por meses de 2021-01-01 a 2022-12-31…
  -> 2021-01-01 a 2021-01-31


2025-09-25 12:02:01,937 INFO Request ID is a9e702ac-fea2-464f-a9d0-ff037d215535
2025-09-25 12:02:02,513 INFO status has been updated to accepted
2025-09-25 12:02:21,253 INFO status has been updated to running
2025-09-25 12:02:41,691 INFO status has been updated to successful


141d12c9e26197832bedf3f3ceff77a.zip:   0%|          | 0.00/45.6k [00:00<?, ?B/s]

2025-09-25 12:02:58,292 INFO Request ID is 4e60ee15-0c1f-46a7-9260-66066cfb87e1
2025-09-25 12:02:58,386 INFO status has been updated to accepted
2025-09-25 12:03:09,925 INFO status has been updated to running
2025-09-25 12:03:23,038 INFO status has been updated to successful


99b4e127ff502832f9ec4ab64d151f3e.nc:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

...

2025-09-25 12:58:47,842 INFO Request ID is 1411ee68-9dfe-4d78-bc4f-25c4a1d73c53
2025-09-25 12:58:47,933 INFO status has been updated to accepted
2025-09-25 12:59:01,727 INFO status has been updated to running
2025-09-25 12:59:20,889 INFO status has been updated to successful


73cee589eb2113e74d7122fd1d4df2da.nc:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

2025-09-25 12:59:22,234 INFO Request ID is 622583c5-64ef-44b1-b3cd-4926ffb5930e
2025-09-25 12:59:22,304 INFO status has been updated to accepted
2025-09-25 12:59:35,902 INFO status has been updated to running
2025-09-25 12:59:55,367 INFO status has been updated to successful


d2523aac830cd3b7303f0e31aec6aacc.nc:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

✔ Insertadas/actualizadas 2190 filas en climate_wxt2 (DB: C:\Users\Nainh\Proton Drive\nainho1306\My files\Licitacion\Plataforma prevencion Dengue\backend\data\mi_base_de_datos5.db)


In [10]:

# Conectar a la base
conn = sqlite3.connect(DB_PATH)

# Leer primeros 20 registros ordenados por fecha
query = '''
SELECT *
FROM climate_wxt2
ORDER BY date ASC
'''

df_head = pd.read_sql_query(query, conn)
conn.close()

df_head["date"].unique()


array(['2021-01-01', '2021-01-02', '2021-01-03', ..., '2025-08-26',
       '2025-08-27', '2025-08-28'], shape=(1701,), dtype=object)

In [11]:
import sqlite3
import pandas as pd


conn = sqlite3.connect(DB_PATH)

query = """
SELECT *
FROM climate_wxt2
ORDER BY date DESC;
"""
df = pd.read_sql(query, conn)

display(df)   # si usas Jupyter
conn.close

,date,latitude,longitude,t2m_min,t2m_max,ws10
0,2025-08-28,-32.0,-61.00,283.687927,297.152588,2.464709
1,2025-08-28,-32.0,-61.25,282.680115,297.211182,2.532038
2,2025-08-28,-32.0,-61.50,282.285583,297.222900,2.645121
3,2025-08-28,-32.0,-61.75,282.758240,297.195557,2.797000
4,2025-08-28,-32.0,-62.00,282.887146,297.533447,2.818601
...,...,...,...,...,...,...
44420,2021-01-05,-33.0,-63.25,293.129883,300.813232,4.032999
44421,2021-01-04,-33.0,-63.25,293.124023,305.167480,0.203458
44422,2021-01-03,-33.0,-63.25,293.963867,300.237061,2.242227
44423,2021-01-02,-33.0,-63.25,293.717285,306.614746,3.978747


<function Connection.close()>

In [12]:
from pathlib import Path
import sqlite3
BASE_DIR = Path().resolve()  

DB_PATH = BASE_DIR.parent / "backend" / "data" / "mi_base_de_datos5.db"

conn = sqlite3.connect(DB_PATH)

# 2) Lée los últimos 20 registros (ordenados por fecha decreciente)
df_clima = pd.read_sql_query("""
    SELECT *
      FROM climate_test
     ORDER BY date
""", conn)

conn.close()
df_clima


,date,latitude,longitude,t2m,d2m,tp,t2m_min,t2m_max,wind_speed,ws10
0,2023-01-01,-33.0,-63.00,296.222351,291.317322,0.000448,292.111572,301.740601,None,2.047017
1,2023-01-01,-33.0,-62.75,296.385437,290.895935,0.000464,292.195557,302.031616,None,1.994279
2,2023-01-01,-33.0,-62.50,296.388855,290.684509,0.000487,292.150635,301.816772,None,1.910873
3,2023-01-01,-33.0,-62.25,296.367371,290.656189,0.000407,292.385010,301.488647,None,1.794032
4,2023-01-01,-33.0,-62.00,296.280457,290.852478,0.000307,292.685791,301.119507,None,1.605475
...,...,...,...,...,...,...,...,...,...,...
42305,2025-08-28,-32.0,-62.00,288.465057,280.686890,0.000000,282.887146,297.533447,None,2.818601
42306,2025-08-28,-32.0,-61.75,288.343475,280.871460,0.000000,282.758240,297.195557,None,2.797000
42307,2025-08-28,-32.0,-61.50,288.132050,281.157104,0.000000,282.285583,297.222900,None,2.645121
42308,2025-08-28,-32.0,-61.25,288.270721,281.509644,0.000000,282.680115,297.211182,None,2.532038


In [13]:
DB_PATH = BASE_DIR.parent / "backend" / "data" / "mi_base_de_datos5.db"

conn = sqlite3.connect(DB_PATH)

cur = conn.cursor()
cur.executescript("""
CREATE INDEX IF NOT EXISTS idx_ct_dll  ON climate_test(date, latitude, longitude);
CREATE INDEX IF NOT EXISTS idx_wxt_dll ON climate_wxt2(date, latitude, longitude);
""")

# update: reemplaza solo con valores no nulos de climate_wxt
cur.executescript("""
UPDATE climate_test AS t
SET
  t2m_min = COALESCE((
      SELECT w.t2m_min
      FROM climate_wxt2 w
      WHERE w.date = t.date
        AND w.latitude  = t.latitude
        AND w.longitude = t.longitude
  ), t.t2m_min),
  t2m_max = COALESCE((
      SELECT w.t2m_max
      FROM climate_wxt2 w
      WHERE w.date = t.date
        AND w.latitude  = t.latitude
        AND w.longitude = t.longitude
  ), t.t2m_max),
  ws10 = COALESCE((
      SELECT w.ws10
      FROM climate_wxt2 w
      WHERE w.date = t.date
        AND w.latitude  = t.latitude
        AND w.longitude = t.longitude
  ), t.ws10)
WHERE EXISTS (
  SELECT 1 FROM climate_wxt2 w
  WHERE w.date = t.date
    AND w.latitude  = t.latitude
    AND w.longitude = t.longitude
);
""")
conn.commit()


In [3]:
import cdsapi



TMP_DIR   = "Clima\\tmp_test20d"
CDS_URL   = 'https://cds.climate.copernicus.eu/api'
CDS_KEY   = 'd9af180c-f7f8-4a2e-8029-09018fb8c920'
c = cdsapi.Client(url=CDS_URL, key=CDS_KEY)
# bbox = [N, W, S, E] en grados; ajusta a tu zona
BBOX = [-31.9, -63.5, -32.6, -63.0]  # aprox VM y alrededores

c.retrieve(
    'reanalysis-era5-land',
    {
        'variable': [
            '2m_temperature','2m_dewpoint_temperature',
            'total_precipitation','10m_u_component_of_wind','10m_v_component_of_wind'
        ],
        'year': ['2025'],     # o un rango
        'month': ['01'],
        'day': [f'{d:02d}' for d in range(1,32)],
        'time': [f'{h:02d}:00' for h in range(24)],  # horario
        'area': BBOX,    # N, W, S, E
        'format': 'netcdf',
    },
    'era5land_vm_2024_2025.nc'
)


2025-09-25 13:16:11,184 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-25 13:16:11,185 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-25 13:16:11,402 INFO Request ID is 3781cfd3-7d3d-40b9-90cb-b2091584f2b1
2025-09-25 13:16:11,490 INFO status has been updated to accepted
2025-09-25 13:16:25,333 INFO status has been updated to running


KeyboardInterrupt: 

In [6]:
import os, cdsapi, calendar, time
import xarray as xr
import pandas as pd

CDS_URL   = 'https://cds.climate.copernicus.eu/api'
CDS_KEY   = 'd9af180c-f7f8-4a2e-8029-09018fb8c920'
c = cdsapi.Client(url=CDS_URL, key=CDS_KEY)

BBOX = [-31.9, -63.5, -32.6, -63.0]  # [N, W, S, E] ajusta a tu zona

# Divide variables en 2 grupos para bajar el costo por request
VAR_GROUPS = [
    ['2m_temperature','2m_dewpoint_temperature','total_precipitation'],
    ['10m_u_component_of_wind','10m_v_component_of_wind'],
]

YEARS  = ['2025']                # o varias ['2024','2025',...]
MONTHS = [f'{m:02d}' for m in range(1, 2)]  # o solo los meses que necesites

OUTDIR = 'downloads_era5land_0p1'
os.makedirs(OUTDIR, exist_ok=True)

def retrieve_month(year, month, variables, bbox=BBOX, outdir=OUTDIR, tag='grp'):
    target = os.path.join(outdir, f'era5land_{year}_{month}_{tag}.nc')
    if os.path.exists(target):
        return target
    # días del mes (evita pedir días inexistentes como 31/02)
    ndays = calendar.monthrange(int(year), int(month))[1]
    days = [f'{d:02d}' for d in range(1, ndays+1)]
    req = {
        'variable': variables,
        'year': year,
        'month': month,
        'day': days,
        'time': [f'{h:02d}:00' for h in range(24)],
        'area': bbox,           # [N, W, S, E]
        'format': 'netcdf',
    }
    # pequeño backoff por si la cola está cargada
    for attempt in range(3):
        try:
            c.retrieve('reanalysis-era5-land', req, target)
            break
        except Exception as e:
            if attempt == 2:
                raise
            time.sleep(15*(attempt+1))
    return target

# === 1) DESCARGA EN LOTES (por mes y grupo de variables) ===
files = []
for y in YEARS:
    for m in MONTHS:
        for gi, group in enumerate(VAR_GROUPS):
            print(m,y,group)
            f = retrieve_month(y, m, group, tag=f'g{gi}')
            files.append(f)

# === 2) ABRE Y COMBINA (concatena a lo largo del tiempo y une grupos por coords) ===
# Si haces 2 grupos por mes, puedes primero unir por mes y luego concatenar meses:
def open_pair(year, month):
    f1 = os.path.join(OUTDIR, f'era5land_{year}_{month}_g0.nc')
    f2 = os.path.join(OUTDIR, f'era5land_{year}_{month}_g1.nc')
    ds1 = xr.open_dataset(f1)
    ds2 = xr.open_dataset(f2)
    return xr.merge([ds1, ds2], compat='override')

datasets = [open_pair(y, m) for y in YEARS for m in MONTHS if 
            os.path.exists(os.path.join(OUTDIR, f'era5land_{y}_{m}_g0.nc')) and
            os.path.exists(os.path.join(OUTDIR, f'era5land_{y}_{m}_g1.nc'))]

ds_hourly = xr.concat(datasets, dim='time').sortby('time')

# === 3) CONVERSIÓN DE UNIDADES Y DIARIO ===
ds_hourly = ds_hourly.rename({
    '2m_temperature':'t2m', '2m_dewpoint_temperature':'d2m',
    'total_precipitation':'tp', '10m_u_component_of_wind':'u10',
    '10m_v_component_of_wind':'v10'
})
ds_hourly['t2m'] = ds_hourly['t2m'] - 273.15
ds_hourly['d2m'] = ds_hourly['d2m'] - 273.15
ds_hourly['ws10'] = (ds_hourly['u10']**2 + ds_hourly['v10']**2)**0.5

daily = xr.Dataset({
    't2m':  ds_hourly['t2m'].resample(time='1D').mean(),
    'd2m':  ds_hourly['d2m'].resample(time='1D').mean(),
    'ws10': ds_hourly['ws10'].resample(time='1D').mean(),
    'tp':   ds_hourly['tp'].resample(time='1D').sum()
})

df_grid = daily.to_dataframe().reset_index().dropna()
df_grid = df_grid.rename(columns={'time':'date', 'latitude':'latitude', 'longitude':'longitude'})
print(df_grid.head())


2025-09-25 13:25:11,047 INFO [2025-09-03T00:00:00] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.
2025-09-25 13:25:11,047 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-09-25 13:25:11,243 INFO Request ID is 99524e1e-a4bf-4609-91e6-ebd68e672fb9


01 2025 ['2m_temperature', '2m_dewpoint_temperature', 'total_precipitation']


2025-09-25 13:25:11,312 INFO status has been updated to accepted
2025-09-25 13:37:31,541 INFO status has been updated to successful


685c57b0271fe5b4993d5d8e2e59eb79.zip:   0%|          | 0.00/240k [00:00<?, ?B/s]

2025-09-25 13:37:32,583 INFO Request ID is fc44ca30-3cdf-483e-9022-01b2ae31168f


01 2025 ['10m_u_component_of_wind', '10m_v_component_of_wind']


2025-09-25 13:37:32,685 INFO status has been updated to accepted
2025-09-25 13:37:41,258 INFO status has been updated to running
2025-09-25 13:41:52,636 INFO status has been updated to successful


83187a3cffe3c6871613b2a5bb4911b4.zip:   0%|          | 0.00/233k [00:00<?, ?B/s]

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html